Mounting Google Drive

In [1]:
#Import Python Packages
#from google.colab import drive
#drive.mount('/content/drive/')

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


Importing Necessary Modules

In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree, metrics
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.neural_network import MLPClassifier
!pip install sklearn-genetic
!pip install sklearn-genetic-opt
import sklearn_genetic
from sklearn_genetic import GASearchCV
from sklearn_genetic import ExponentialAdapter
from sklearn_genetic.space import Continuous, Categorical, Integer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#Read files
trainfile = r'/gdrive/My Drive/A5/train.csv'
trainData = pd.read_csv(trainfile) #creates a dataframe

testfile = r'/gdrive/My Drive/A5/test.csv'
testData = pd.read_csv(testfile)  #creates a dataframe

print(trainData.shape)
print(testData.shape)

(76020, 371)
(75818, 370)


**EDA**

In [4]:
trainData.info()
print()
testData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75818 entries, 0 to 75817
Columns: 370 entries, ID to var38
dtypes: float64(110), int64(260)
memory usage: 214.0 MB


In [5]:
trainData.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


In [6]:
#To get list of names of all Columns from a dataframe

TrainCols = list(trainData.columns.values)
TestCols = list(testData.columns.values)
print(TrainCols)
print(TestCols)

['ID', 'var3', 'var15', 'imp_ent_var16_ult1', 'imp_op_var39_comer_ult1', 'imp_op_var39_comer_ult3', 'imp_op_var40_comer_ult1', 'imp_op_var40_comer_ult3', 'imp_op_var40_efect_ult1', 'imp_op_var40_efect_ult3', 'imp_op_var40_ult1', 'imp_op_var41_comer_ult1', 'imp_op_var41_comer_ult3', 'imp_op_var41_efect_ult1', 'imp_op_var41_efect_ult3', 'imp_op_var41_ult1', 'imp_op_var39_efect_ult1', 'imp_op_var39_efect_ult3', 'imp_op_var39_ult1', 'imp_sal_var16_ult1', 'ind_var1_0', 'ind_var1', 'ind_var2_0', 'ind_var2', 'ind_var5_0', 'ind_var5', 'ind_var6_0', 'ind_var6', 'ind_var8_0', 'ind_var8', 'ind_var12_0', 'ind_var12', 'ind_var13_0', 'ind_var13_corto_0', 'ind_var13_corto', 'ind_var13_largo_0', 'ind_var13_largo', 'ind_var13_medio_0', 'ind_var13_medio', 'ind_var13', 'ind_var14_0', 'ind_var14', 'ind_var17_0', 'ind_var17', 'ind_var18_0', 'ind_var18', 'ind_var19', 'ind_var20_0', 'ind_var20', 'ind_var24_0', 'ind_var24', 'ind_var25_cte', 'ind_var26_0', 'ind_var26_cte', 'ind_var26', 'ind_var25_0', 'ind_var2

In [7]:
#Removing 'Target' column

Xtrain = trainData[TrainCols[0:len(TrainCols)-1]].copy()
Ytrain = trainData[['TARGET']].copy()
print(Xtrain.shape)
print(Ytrain.shape)
Xtest = testData.copy()

(76020, 370)
(76020, 1)


Printing the distribution of target class values

In [8]:
trainData["TARGET"].value_counts()

0    73012
1     3008
Name: TARGET, dtype: int64

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(Xtrain, Ytrain, test_size = 0.2 , random_state = 1, stratify = Ytrain)

In [10]:
Y_test.shape

(15204, 1)

To handle the under-representation of the target class, we will perform either:


*  Undersampling
*  Oversampling





In [11]:
# instantiating the random undersampler
from imblearn.under_sampling import RandomUnderSampler
U = RandomUnderSampler(sampling_strategy= 1) 
# resampling X, y
X_u, Y_u = U.fit_resample(X_train, Y_train)
# new class distribution
X_u.shape

(4812, 370)

In [12]:
Y_u["TARGET"].value_counts()

0    2406
1    2406
Name: TARGET, dtype: int64

In [13]:
Y_u.shape

(4812, 1)

In [14]:
X_test.shape

(15204, 370)

In [15]:
Y_test.shape

(15204, 1)

Creating a decision tree classifier and training it
Changes made over a default classifier:


*   Splitting criterion changed to "Entropy" instead of "Gini"
*   Max number of leaf nodes changed to 20



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 139 kB 26.1 MB/s 
     |████████████████████████████████| 115 kB 78.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import matplotlib.pyplot as plt
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Categorical, Integer, Continuous
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_digits
from sklearn.metrics import accuracy_score

In [ ]:
#GENETIC ALGORITHM FOR FEATURE SELECTION 25 ===================================================================================
#===========================================================================================================================
#https://pypi.org/project/sklearn-genetic/

from genetic_selection import GeneticSelectionCV

x = X_u.iloc[:,:-1].values
y = Y_u.iloc[:].values

clf = DecisionTreeClassifier()
model = GeneticSelectionCV(
    clf, cv=5, verbose=0,
    scoring="accuracy", max_features=25,
    n_population=100, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model = model.fit(x, y)

#Do feature selection using the genetic algo
#Get column names
print(model.get_support())

#Get column names
cols = model.get_support(indices=True) #get column indices
print("\n cols = ", cols, "\n")

features_df_new_ga = X_train.iloc[:,cols] #Get cols of X-Train with col names
print(features_df_new_ga.shape)
print ("\n features_df_new_ga= ", features_df_new_ga)
#print(features_df_new.head())

GA_25 = cols

/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
features_df_new_ga

,var3,var15,imp_op_var41_ult1,ind_var8,ind_var13_corto_0,ind_var13_largo,ind_var13_medio,ind_var13,ind_var20_0,ind_var30_0,...,imp_amort_var34_hace3,num_op_var39_comer_ult1,num_op_var40_comer_ult1,num_trasp_var33_in_hace3,saldo_medio_var5_ult1,saldo_medio_var12_ult3,saldo_medio_var29_hace2,saldo_medio_var29_ult1,saldo_medio_var33_hace2,saldo_medio_var33_hace3
18511,2,28,0.00,0,0,0,0,0,0,1,...,0,0,0,0,0.00,0.0,0.0,0.0,0.0,0.0
35536,2,23,0.00,0,0,0,0,0,0,1,...,0,0,0,0,0.00,0.0,0.0,0.0,0.0,0.0
70256,2,24,0.00,0,0,0,0,0,0,1,...,0,0,0,0,3.00,0.0,0.0,0.0,0.0,0.0
44704,2,42,153.45,0,0,0,0,0,0,1,...,0,6,0,0,928.14,0.0,0.0,0.0,0.0,0.0
43542,2,62,0.00,0,0,0,0,0,0,1,...,0,0,0,0,15.00,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26903,2,26,0.00,0,0,0,0,0,0,1,...,0,0,0,0,0.00,0.0,0.0,0.0,0.0,0.0
3965,2,47,0.00,0,0,0,0,0,0,1,...,0,0,0,0,120.00,0.0,0.0,0.0,0.0,0.0
47917,2,35,0.00,0,0,0,0,0,0,1,...,0,0,0,0,15.36,0.0,0.0,0.0,0.0,0.0
14037,2,27,0.00,0,0,0,0,0,0,1,...,0,0,0,0,0.00,0.0,0.0,0.0,0.0,0.0


In [ ]:
#GENETIC ALGORITHM FOR FEATURE SELECTION 50 ===================================================================================
#===========================================================================================================================
#https://pypi.org/project/sklearn-genetic/

from genetic_selection import GeneticSelectionCV

x = X_u.iloc[:,:-1].values
y = Y_u.iloc[:].values

clf = DecisionTreeClassifier()
model = GeneticSelectionCV(
    clf, cv=5, verbose=0,
    scoring="accuracy", max_features=50,
    n_population=100, crossover_proba=0.5,
    mutation_proba=0.2, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model = model.fit(x, y)

#Do feature selection using the genetic algo
#Get column names
print(model.get_support())

#Get column names
cols = model.get_support(indices=True) #get column indices
print("\n cols = ", cols, "\n")

features_df_new_ga = X_train.iloc[:,cols] #Get cols of X-Train with col names
print(features_df_new_ga.shape)
print ("\n features_df_new_ga= ", features_df_new_ga)
#print(features_df_new.head())
GA_50 = cols

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[False  True  True False False False False False False False False False
 False False False  True False False False False False False False False
 False False False False False  True False False False  True False False
  True False  True  True False False False False False False False  True
 False False False False False False False False False False False False
 False False False  True False False False  True False False False False
 False False False False False False False False  True False False False
  True False False False False False  True False False False False False
 False False False False False False False False False  True False False
 False False  True False False False False False  True False False False
 False False False False False False  True False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False  True
 False False False False False False False False  T

In [ ]:

#K-BEST FEATURE Selection =================================================================================
#====================================================================================================
#Feature Selection
fs = SelectKBest(f_classif, k=25)
#fs = SelectKBest(chi2, k=10)
fs.fit(X_u, Y_u)
SelectKBest_25 = fs.get_support(True)
print ("25 Important Features", SelectKBest_25)


#f_value, p_value = f_classif(X_train, y_train)
#print (feature_index.tolist(), f_value, p_value) 

25 Important Features [  2  25  30  31  32  33  34  39  64  89  91  97  98  99 100 105 139 148
 159 172 183 194 281 284 369]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 22  23  26  27  37  38  44  45  57  58  59  60  61  62  81  85  86  92
  93 103 104 110 111 132 133 134 135 136 137 156 162 163 166 171 175 180
 181 182 190 193 195 201 202 203 204 205 206 207 213 214 215 216 217 218
 219 221 222 223 227 230 231 235 236 237 238 239 240 243 244 245 246 247
 248 249 250 251 253 262 263 266 269 270 286 288 304 305 306 307 308 309
 314 315 316 317 318 319 320 321 322 328 349 350 351 352 354 357 358 359
 360] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selec

In [ ]:

#K-BEST FEATURE Selection of 50 features=================================================================================
#====================================================================================================
#Feature Selection
fs = SelectKBest(f_classif, k=50)
#fs = SelectKBest(chi2, k=10)
fs.fit(X_u, Y_u)
SelectKBest_50 = fs.get_support(True)
print ("50 Important Features", SelectKBest_50)

#f_value, p_value = f_classif(X_train, y_train)
#print (feature_index.tolist(), f_value, p_value) 

50 Important Features [  2  13  16  25  28  30  31  32  33  34  35  36  39  49  50  64  89  91
  94  97  98  99 100 105 114 115 139 148 159 168 169 172 177 183 191 194
 225 264 281 283 284 329 330 332 339 340 341 343 344 369]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 22  23  26  27  37  38  44  45  57  58  59  60  61  62  81  85  86  92
  93 103 104 110 111 132 133 134 135 136 137 156 162 163 166 171 175 180
 181 182 190 193 195 201 202 203 204 205 206 207 213 214 215 216 217 218
 219 221 222 223 227 230 231 235 236 237 238 239 240 243 244 245 246 247
 248 249 250 251 253 262 263 266 269 270 286 288 304 305 306 307 308 309
 314 315 316 317 318 319 320 321 322 328 349 350 351 352 354 357 358 359
 360] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selec

In [ ]:
common_features_25 = []
common_features_50 = []
for i in SelectKBest_25:
  if i in GA_25:
    common_features_25.append(i)
for i in SelectKBest_50:
  if i in GA_50:
    common_features_50.append(i)
print(common_features_25, "\n", common_features_50)


[2, 30] 
 [2, 33, 36, 39, 105, 191, 340]


In [ ]:
# Getting only common columns
X_rus_common25 = X_u.iloc[:,[2,91,139]]
X_rus_common50 = X_u.iloc[:, [2, 13, 16, 18, 32, 99, 105, 183, 191, 194]]


In [ ]:
X_rus_common25 = X_u.iloc[:, common_features_25]
X_rus_common50 = X_u.iloc[:, common_features_50]
X_test25 = X_test.iloc[:,common_features_25]
X_test50 = X_test.iloc[:,common_features_50]

In [ ]:
X_rus_common50

,var15,ind_var13_corto_0,ind_var13_largo,ind_var13,num_var13,saldo_var42,saldo_medio_var12_ult3
0,23,0,0,0,0,0.00,0.0
1,24,0,0,0,0,0.00,0.0
2,24,0,0,0,0,3.00,0.0
3,23,0,0,0,0,3.00,0.0
4,26,0,0,0,0,15.00,0.0
...,...,...,...,...,...,...,...
4807,35,0,0,0,0,63.00,0.0
4808,24,0,0,0,0,150.00,0.0
4809,38,0,0,0,0,0.00,0.0
4810,47,0,0,0,0,1137.69,0.0


# Random Forest Classifier with top 25 features and GA Hyperparameter tuning

In [ ]:
# Top 25 RF Classifier
clf = RandomForestClassifier()

#mutation_adapter = ExponentialAdapter(initial_value=0.8, end_value=0.2, adaptive_rate=0.1)
#crossover_adapter = ExponentialAdapter(initial_value=0.2, end_value=0.8, adaptive_rate=0.1)

param_grid = {'min_weight_fraction_leaf': Continuous(0.01, 0.5, distribution='log-uniform'),
              'bootstrap': Categorical([True, False]),
              'max_depth': Integer(2, 30),
              'max_leaf_nodes': Integer(2, 35),
              'n_estimators': Integer(100, 300)}

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=clf,
                               cv=cv,
                               scoring='accuracy',
                               population_size=20,
                               generations=25,
                              #  mutation_probability=mutation_adapter,
                              #  crossover_probability=crossover_adapter,
                               param_grid=param_grid,
                               n_jobs=-1)

# Train and optimize the estimator
evolved_estimator.fit(X_rus_common25, Y_u)
# Best parameters found
print(evolved_estimator.best_params_)
# Use the model fitted with the best parameters
y_predict_ga = evolved_estimator.predict(X_test25)
print(accuracy_score(Y_test, y_predict_ga))

# Saved metadata for further analysis
print("Stats achieved in each generation: ", evolved_estimator.history)
print("Best k solutions: ", evolved_estimator.hof)

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.682149	0.0168115  	0.70345    	0.649418   
1  	40    	0.698005	0.00787162 	0.70345    	0.681629   
2  	40    	0.702442	0.00369668 	0.70345    	0.686409   
3  	40    	0.70345 	2.48253e-17	0.70345    	0.70345    
4  	40    	0.702961	0.00123693 	0.70345    	0.69867    
5  	40    	0.70345 	6.08094e-17	0.70345    	0.70345    
6  	40    	0.70345 	6.56817e-17	0.70345    	0.70345    
7  	40    	0.70345 	7.02167e-17	0.70345    	0.70345    
8  	40    	0.703304	0.000634088	0.70345    	0.70054    
9  	40    	0.70345 	6.08094e-17	0.70345    	0.70345    
10 	40    	0.702307	0.00412001 	0.70345    	0.684539   
11 	40    	0.703367	0.000362336	0.70345    	0.701787   
12 	40    	0.703283	0.000724671	0.70345    	0.700125   
13 	40    	0.70345 	7.4476e-17 	0.70345    	0.70345    
14 	40    	0.70345 	7.4476e-17 	0.70345    	0.70345    
15 	40    	0.70345 	7.85046e-17	0.70345    	0.70345    
16 	40    	0.7032  	0.00108701 	0.70345    	0.69

/usr/local/lib/python3.7/dist-packages/sklearn_genetic/genetic_search.py:555: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator.fit(self.X_, self.y_)


{'min_weight_fraction_leaf': 0.022518933603851782, 'bootstrap': True, 'max_depth': 4, 'max_leaf_nodes': 17, 'n_estimators': 131}
0.5880689292291502
Stats achieved in each generation:  {'gen': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25], 'fitness': [0.6821487946799667, 0.698004987531172, 0.7024418121363258, 0.7034497090606816, 0.7029613466334164, 0.7034497090606816, 0.7034497090606816, 0.7034497090606816, 0.7033042394014963, 0.7034497090606816, 0.7023067331670821, 0.7033665835411471, 0.7032834580216126, 0.7034497090606816, 0.7034497090606816, 0.7034497090606816, 0.7032003325020781, 0.7034497090606816, 0.7034497090606816, 0.7034497090606816, 0.7034497090606816, 0.7033458021612635, 0.7033458021612635, 0.7033665835411471, 0.7034497090606816, 0.7034497090606816], 'fitness_std': [0.016811545497500798, 0.007871618024776744, 0.003696676880201962, 2.482534153247273e-17, 0.001236928609638858, 6.080941944488117e-17, 6.568167990716596e-17, 7.02166

In [ ]:
tuned_params = evolved_estimator.best_params_
print(tuned_params)

{'min_weight_fraction_leaf': 0.022518933603851782, 'bootstrap': True, 'max_depth': 4, 'max_leaf_nodes': 17, 'n_estimators': 131}


In [ ]:


kaggle_test25 = testData.iloc[:, common_features_25]
kaggle_test50 = testData.iloc[:, common_features_50]

In [ ]:


X_train_25_whole = X_train.iloc[:, common_features_25]
X_train_50_whole = X_train.iloc[:, common_features_50]

In [ ]:
#Using the parameters obtained from HyperParameterTuning in the RandomForestClassifier
clf = RandomForestClassifier(**tuned_params, class_weight = "balanced")
clf.fit(X_train_25_whole, Y_train)
clf_predict = clf.predict(kaggle_test25)
submission = Xtest[["ID"]]
submission["TARGET"]= clf_predict
submission.to_csv("/gdrive/My Drive/A5/results_c.csv", index = None)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Random Forest Classifier with top 50 features and HPT

In [ ]:
# Top 50 RF Classifier
clf = RandomForestClassifier()

#mutation_adapter = ExponentialAdapter(initial_value=0.8, end_value=0.2, adaptive_rate=0.1)
#crossover_adapter = ExponentialAdapter(initial_value=0.2, end_value=0.8, adaptive_rate=0.1)

param_grid = {'min_weight_fraction_leaf': Continuous(0.01, 0.5, distribution='log-uniform'),
              'bootstrap': Categorical([True, False]),
              'max_depth': Integer(2, 30),
              'max_leaf_nodes': Integer(2, 35),
              'n_estimators': Integer(100, 300)}

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=clf,
                               cv=cv,
                               scoring='accuracy',
                               population_size=20,
                               generations=25,
                              #  mutation_probability=mutation_adapter,
                              #  crossover_probability=crossover_adapter,
                               param_grid=param_grid,
                               n_jobs=-1)

# Train and optimize the estimator
evolved_estimator.fit(X_rus_common50, Y_u)
# Best parameters found
print(evolved_estimator.best_params_)
# Use the model fitted with the best parameters
y_predict_ga = evolved_estimator.predict(X_test50)
print(accuracy_score(Y_test, y_predict_ga))

# Saved metadata for further analysis
print("Stats achieved in each generation: ", evolved_estimator.history)
print("Best k solutions: ", evolved_estimator.hof)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.717041	0.0275328  	0.751663   	0.649418   
1  	40    	0.741033	0.0096539  	0.752078   	0.720698   
2  	40    	0.747693	0.00453391 	0.752078   	0.73192    
3  	40    	0.748899	0.00231047 	0.752078   	0.742311   
4  	40    	0.748951	0.00192323 	0.751247   	0.743973   
5  	40    	0.749678	0.00167735 	0.751455   	0.744389   
6  	40    	0.75001 	0.00111617 	0.751455   	0.747714   
7  	40    	0.750052	0.00147807 	0.751455   	0.746259   
8  	40    	0.750156	0.00132068 	0.751455   	0.746675   
9  	40    	0.750436	0.00157569 	0.752078   	0.746675   
10 	40    	0.750249	0.00221065 	0.754988   	0.74522    
11 	40    	0.748732	0.00508184 	0.752078   	0.728387   
12 	40    	0.74867 	0.00204187 	0.752078   	0.742727   
13 	40    	0.749117	0.00146634 	0.751247   	0.746052   
14 	40    	0.749418	0.00166718 	0.751247   	0.743558   
15 	40    	0.750364	0.00132068 	0.753117   	0.74813    
16 	40    	0.749938	0.00215075 	0.753117   	0.74

/usr/local/lib/python3.7/dist-packages/sklearn_genetic/genetic_search.py:555: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator.fit(self.X_, self.y_)


{'min_weight_fraction_leaf': 0.013398999972304449, 'bootstrap': True, 'max_depth': 16, 'max_leaf_nodes': 33, 'n_estimators': 274}
0.7352012628255722
Stats achieved in each generation:  {'gen': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25], 'fitness': [0.7170407315045719, 0.7410328345802161, 0.7476932668329178, 0.748898586866168, 0.7489505403158769, 0.7496778886118038, 0.7500103906899419, 0.750051953449709, 0.7501558603491272, 0.7504364089775561, 0.7502493765586037, 0.748732335827099, 0.7486699916874481, 0.7491167913549461, 0.7494181213632587, 0.7503636741479635, 0.749937655860349, 0.7496363258520365, 0.7499376558603491, 0.7505610972568578, 0.7510494596841231, 0.7500831255195346, 0.7486596009975064, 0.7496051537822112, 0.7495220282626767, 0.7502078137988363], 'fitness_std': [0.02753281093904922, 0.009653901618113761, 0.004533914184007376, 0.0023104745257440194, 0.0019232322223740092, 0.0016773483221887043, 0.0011161651857723516, 0.0014780

In [ ]:
tuned_params = evolved_estimator.best_params_
print(tuned_params)

{'min_weight_fraction_leaf': 0.013398999972304449, 'bootstrap': True, 'max_depth': 16, 'max_leaf_nodes': 33, 'n_estimators': 274}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the RandomForestClassifier
clf = RandomForestClassifier(**tuned_params, class_weight = "balanced")
clf.fit(X_train_50_whole, Y_train)
clf_predict = clf.predict(kaggle_test50)
submission_50 = Xtest[["ID"]]
submission_50["TARGET"]= clf_predict
submission_50.to_csv("/gdrive/My Drive/A5/results_50.csv", index = None)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Decision Tree Classifier top 25 features Hyperparameter tuning with GA

In [ ]:
# Top 25 RF Classifier
clf = DecisionTreeClassifier()

#mutation_adapter = ExponentialAdapter(initial_value=0.8, end_value=0.2, adaptive_rate=0.1)
#crossover_adapter = ExponentialAdapter(initial_value=0.2, end_value=0.8, adaptive_rate=0.1)

param_grid = {'min_samples_split' : Integer(10,100),'max_depth': Integer(1,20)}

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=clf,
                               cv=cv,
                               scoring='accuracy',
                               population_size=20,
                               generations=25,
                              #  mutation_probability=mutation_adapter,
                              #  crossover_probability=crossover_adapter,
                               param_grid=param_grid,
                               n_jobs=-1)

# Train and optimize the estimator
evolved_estimator.fit(X_rus_common25, Y_u)
# Best parameters found
print(evolved_estimator.best_params_)
# Use the model fitted with the best parameters
y_predict_ga = evolved_estimator.predict(X_test25)
print(accuracy_score(Y_test, y_predict_ga))

# Saved metadata for further analysis
print("Stats achieved in each generation: ", evolved_estimator.history)
print("Best k solutions: ", evolved_estimator.hof)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.698286	0.00222785 	0.702618   	0.694929   
1  	40    	0.701725	0.0011135  	0.70345    	0.69867    
2  	40    	0.701735	0.000704348	0.703034   	0.700333   
3  	40    	0.702161	0.000586315	0.703034   	0.700333   
4  	40    	0.701486	0.00172432 	0.702826   	0.696592   
5  	40    	0.702078	0.0011578  	0.70345    	0.698254   
6  	40    	0.702546	0.000831969	0.70345    	0.699917   
7  	40    	0.702889	0.00105167 	0.70345    	0.699501   
8  	40    	0.702577	0.000890452	0.70345    	0.700748   
9  	40    	0.703055	0.000598345	0.70345    	0.700748   
10 	40    	0.702857	0.00100586 	0.70345    	0.699293   
11 	40    	0.703096	0.000591813	0.70345    	0.700748   
12 	40    	0.7032  	0.00051578 	0.70345    	0.701164   
13 	40    	0.703242	0.000347739	0.70345    	0.701995   
14 	40    	0.703096	0.000657493	0.70345    	0.700956   
15 	40    	0.70318 	0.000577034	0.70345    	0.700956   
16 	40    	0.702847	0.00116431 	0.70345    	0.70

In [ ]:
tuned_params = evolved_estimator.best_params_
print(tuned_params)

{'min_samples_split': 36, 'max_depth': 4}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the DTClassifier
clf = DecisionTreeClassifier(**tuned_params, class_weight = "balanced")
clf.fit(X_train_25_whole, Y_train)
clf_predict = clf.predict(kaggle_test25)
submission_25 = Xtest[["ID"]]
submission_25["TARGET"]= clf_predict
submission_25.to_csv("/gdrive/My Drive/A5/resultsDT_25.csv", index = None)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Decision Tree Classifier Top 50 features hyperparameter tuning using GA

In [ ]:
# Top 50 DT Classifier
clf = DecisionTreeClassifier()

#mutation_adapter = ExponentialAdapter(initial_value=0.8, end_value=0.2, adaptive_rate=0.1)
#crossover_adapter = ExponentialAdapter(initial_value=0.2, end_value=0.8, adaptive_rate=0.1)

param_grid = {'min_samples_split' : Integer(10,100),'max_depth': Integer(1,20)}

cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=clf,
                               cv=cv,
                               scoring='accuracy',
                               population_size=20,
                               generations=25,
                              #  mutation_probability=mutation_adapter,
                              #  crossover_probability=crossover_adapter,
                               param_grid=param_grid,
                               n_jobs=-1)

# Train and optimize the estimator
evolved_estimator.fit(X_rus_common50, Y_u)
# Best parameters found
print(evolved_estimator.best_params_)
# Use the model fitted with the best parameters
y_predict_ga = evolved_estimator.predict(X_test50)
print(accuracy_score(Y_test, y_predict_ga))

# Saved metadata for further analysis
print("Stats achieved in each generation: ", evolved_estimator.history)
print("Best k solutions: ", evolved_estimator.hof)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.731193	0.00845843 	0.744181   	0.7101     
1  	40    	0.737604	0.00379183 	0.743766   	0.731712   
2  	40    	0.74016 	0.00166157 	0.743766   	0.736284   
3  	40    	0.741823	0.00211447 	0.747506   	0.737947   
4  	40    	0.742508	0.0023248  	0.747506   	0.737947   
5  	40    	0.742467	0.00256452 	0.747506   	0.735661   
6  	40    	0.742488	0.00263627 	0.747506   	0.736908   
7  	40    	0.743049	0.00386862 	0.747506   	0.729426   
8  	40    	0.743236	0.00291197 	0.747506   	0.734622   
9  	40    	0.742342	0.00368439 	0.747506   	0.733167   
10 	40    	0.744566	0.00193866 	0.747506   	0.740441   
11 	40    	0.744057	0.00367566 	0.747506   	0.734206   
12 	40    	0.744618	0.0021755  	0.747091   	0.740441   
13 	40    	0.744503	0.00233129 	0.747091   	0.740441   
14 	40    	0.744534	0.00273818 	0.74813    	0.737531   
15 	40    	0.744503	0.00342702 	0.74813    	0.732959   
16 	40    	0.744462	0.00268255 	0.748337   	0.73

In [ ]:
tuned_params = evolved_estimator.best_params_
print(tuned_params)

{'min_samples_split': 100, 'max_depth': 5}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the RandomForestClassifier
clf = DecisionTreeClassifier(**tuned_params, class_weight = "balanced")
clf.fit(X_train_50_whole, Y_train)
clf_predict = clf.predict(kaggle_test50)
submissiondt_50 = Xtest[["ID"]]
submissiondt_50["TARGET"]= clf_predict
submissiondt_50.to_csv("/gdrive/My Drive/A5/resultsdt_50.csv", index = None)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## MLP Classifier with top 25 features and GA

In [ ]:
import itertools

In [ ]:
# # Top 25 MLP Classifier
# clf = MLPClassifier()

# #mutation_adapter = ExponentialAdapter(initial_value=0.8, end_value=0.2, adaptive_rate=0.1)
# #crossover_adapter = ExponentialAdapter(initial_value=0.2, end_value=0.8, adaptive_rate=0.1)

# param_grid = { 'activation':['tanh', 'relu'], 'learning_rate':['constant', 'adaptive'], 'max_iter' :[100, 150]}
# cv = StratifiedKFold(n_splits=3, shuffle=True)

# evolved_estimator = GASearchCV(estimator=clf,
#                                cv=cv,
#                                scoring='accuracy',
#                                population_size=20,
#                                generations=25,
#                               #  mutation_probability=mutation_adapter,
#                               #  crossover_probability=crossover_adapter,
#                                param_grid=param_grid,
#                                n_jobs=-1)

# # Train and optimize the estimator
# evolved_estimator.fit(X_rus_common25, Y_rus)
# # Best parameters found
# print(evolved_estimator.best_params_)
# # Use the model fitted with the best parameters
# y_predict_ga = evolved_estimator.predict(X_test25)
# print(accuracy_score(Y_test, y_predict_ga))

# # Saved metadata for further analysis
# print("Stats achieved in each generation: ", evolved_estimator.history)
# print("Best k solutions: ", evolved_estimator.hof)


param_grid = {'tol': Continuous(1e-2, 1e10, distribution='log-uniform'),
              'alpha': Continuous(1e-5, 2e-5),
              'activation': Categorical(['logistic', 'tanh']),
              'batch_size': Integer(300, 350)
             }
# The base classifier to tune
clf = MLPClassifier(hidden_layer_sizes=(50, 30))
# Our cross-validation strategy (it could be just an int)
cv = StratifiedKFold(n_splits=3, shuffle=True)
# The main class from sklearn-genetic-opt
evolved_estimator = GASearchCV(estimator=clf,
                              cv=cv,
                              scoring='accuracy',
                              param_grid=param_grid,
                              n_jobs=-1,
                              verbose=True,
                              population_size=20,
                              generations=25)
# Train and optimize the estimator
evolved_estimator.fit(X_rus_common25, Y_u)
# Best parameters found
print(evolved_estimator.best_params_)
# Use the model fitted with the best parameters
y_predict_ga = evolved_estimator.predict(X_test25)
print(accuracy_score(Y_test, y_predict_ga))

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.658365	0.00843158 	0.674148   	0.643599   
1  	40    	0.66223 	0.00565096 	0.674148   	0.651288   
2  	40    	0.665846	0.00575741 	0.678512   	0.656276   
3  	40    	0.668381	0.00528308 	0.678512   	0.658354   
4  	40    	0.667363	0.00612338 	0.678512   	0.652743   
5  	40    	0.665357	0.00270086 	0.669992   	0.658354   
6  	40    	0.665659	0.00375751 	0.669992   	0.657315   
7  	40    	0.668402	0.00459606 	0.67394    	0.658146   
8  	40    	0.669711	0.00363126 	0.67394    	0.660848   
9  	40    	0.668869	0.0053309  	0.675603   	0.660224   
10 	40    	0.669545	0.00562184 	0.675603   	0.656692   
11 	40    	0.669981	0.0055434  	0.675603   	0.656484   
12 	40    	0.669586	0.00647282 	0.675603   	0.656692   
13 	40    	0.670418	0.00555585 	0.676642   	0.656068   
14 	40    	0.670833	0.00532728 	0.676642   	0.661264   
15 	40    	0.67154 	0.00563956 	0.676642   	0.657523   
16 	40    	0.670137	0.00537796 	0.677889   	0.66

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'tol': 2166195.086020421, 'alpha': 1.3898310408198733e-05, 'activation': 'tanh', 'batch_size': 320}
0.6417390160484083


In [ ]:
tuned_params = evolved_estimator.best_params_
print(tuned_params)

{'tol': 2166195.086020421, 'alpha': 1.3898310408198733e-05, 'activation': 'tanh', 'batch_size': 320}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the MLPClassifier
clf = MLPClassifier(**tuned_params)
clf.fit(X_train_25_whole, Y_train)
clf_predict = clf.predict(kaggle_test25)
submission = Xtest[["ID"]]
submission["TARGET"]= clf_predict
submission.to_csv("/gdrive/My Drive/A5/resultmlp_25.csv", index = None)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## MLP Classifier with top 50 features and GA

In [ ]:
param_grid = {'tol': Continuous(1e-2, 1e10, distribution='log-uniform'),
              'alpha': Continuous(1e-5, 2e-5),
              'activation': Categorical(['logistic', 'tanh']),
              'batch_size': Integer(300, 350)
             }
# The base classifier to tune
clf = MLPClassifier(hidden_layer_sizes=(50, 30))
# Our cross-validation strategy (it could be just an int)
cv = StratifiedKFold(n_splits=3, shuffle=True)
# The main class from sklearn-genetic-opt
evolved_estimator = GASearchCV(estimator=clf,
                              cv=cv,
                              scoring='accuracy',
                              param_grid=param_grid,
                              n_jobs=-1,
                              verbose=True,
                              population_size=20,
                              generations=25)
# Train and optimize the estimator
evolved_estimator.fit(X_rus_common50, Y_u)
# Best parameters found
print(evolved_estimator.best_params_)
# Use the model fitted with the best parameters
y_predict_ga = evolved_estimator.predict(X_test50)
print(accuracy_score(Y_test, y_predict_ga))

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.673047	0.0316512  	0.716958   	0.613259   
1  	40    	0.694483	0.0180667  	0.723815   	0.653782   
2  	40    	0.708468	0.00816612 	0.723815   	0.689526   
3  	40    	0.710505	0.00669453 	0.722153   	0.698047   
4  	40    	0.711866	0.0102883  	0.72714    	0.694098   
5  	40    	0.714266	0.00946876 	0.72714    	0.689318   
6  	40    	0.714017	0.0126173  	0.72714    	0.685786   
7  	40    	0.714973	0.00753521 	0.721945   	0.693267   
8  	40    	0.715222	0.00749349 	0.721945   	0.696592   
9  	40    	0.718277	0.00549677 	0.72714    	0.702826   
10 	40    	0.712282	0.0104763  	0.721945   	0.69202    
11 	40    	0.711399	0.00882337 	0.721945   	0.69015    
12 	40    	0.716168	0.00559232 	0.722984   	0.70054    
13 	40    	0.71781 	0.00598233 	0.727972   	0.700956   
14 	40    	0.714225	0.00625387 	0.727972   	0.699917   
15 	40    	0.713071	0.00658317 	0.727972   	0.700125   
16 	40    	0.71383 	0.00771672 	0.727972   	0.69

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'tol': 0.013959777166475703, 'alpha': 1.7414641362016725e-05, 'activation': 'tanh', 'batch_size': 330}
0.7369113391212839


In [ ]:
tuned_params = evolved_estimator.best_params_
print(tuned_params)

{'tol': 0.013959777166475703, 'alpha': 1.7414641362016725e-05, 'activation': 'tanh', 'batch_size': 330}


In [ ]:
#Using the parameters obtained from HyperParameterTuning in the MLPClassifier
clf = MLPClassifier(**tuned_params)
clf.fit(X_train_50_whole, Y_train)
clf_predict = clf.predict(kaggle_test50)
submission = Xtest[["ID"]]
submission["TARGET"]= clf_predict
submission.to_csv("/gdrive/My Drive/A5/resultmlp_50.csv", index = None)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
